In [21]:
import re
import jieba
import random
import time
from collections import defaultdict

In [22]:
# 判断是否为 ？*x 形式
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [23]:
# 判断是否为  ？x 形式
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [24]:
# 如果有多个变量，就将其打包成元组，并将其以列表形式返回

def pat_match(pattern, saying):
    if not pattrn or saying:
        return []
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]:
            return[]
        else:
            return pat_match(pattern[1:], saying[1:])

In [25]:
# 返回两个list内容是否相同
def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    
    return is_match(rest[1:], saying[1:])

In [26]:
# 贪婪匹配

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*','?')
    
    if not rest:
        return (seg_pat, saying), len(saying)
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i+1):]):
            return (seg_pat,saying[:i]), i
        
    if i == len(saying)-1:
        return (), -1
    
    return (seg_pat, saying), len(saying)

In [27]:
# 全局寻找匹配
def to_match(pattern, saying):
    i, j = 0, 0
    m, n = len(pattern), len(saying)
    
    while i < m and j < n:
        if not is_pattern_segment(pattern[i]):
            while j < n:
                if pattern[i] != saying[j]:
                    j += 1
                else:
                    i += 1
                    j += 1
                    if i < m and is_pattern_segment(pattern[i]):
                        break
                        
                    if i < m and not is_pattern_segment(pattern[i]) and pattern[i] != saying[j]:
                        return False
        else:
            i += 1
            
    if i < m and j == n:
        return False
    
    return True

In [28]:
# 单值和多值匹配问题

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying:
        return []
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return []

In [29]:
def subsitite(rule, parsed_rules):
    if not rule:
        return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [30]:
def pat_to_dict1(patterns):
    return {k:v for k, v in patterns}

In [31]:
def pat_to_dict2(pattens):
    return{k:''.join(v) if isinstance(v,list) else v for k,v in pattens}

In [32]:
def cut_chinese(string):

    str_cut = []

    str_split = re.compile(r"(\?\*?[A-Za-z])").split(string)   # 以?*z为分隔符进行切分

    for x in str_split:

        if is_variable(x) or is_pattern_segment(x):  # ?x or ?*x

            str_cut.append(x)

        else:

            str_cut = str_cut + jieba.lcut(x)

    return str_cut


In [33]:
rule_responses = {
    '?*x你好?*y': ['你好，', '有事么'],

    '?*想要?*y': ['现在没有?y，你需要其他东西么？'],

    '?*x难受?*y': ['难受在?y里？', '?y有什么问题么？'],

}

In [34]:
def ask_response():
    ask = input("input:")
    for k,v in rule_responses.items():
        pattern = cut_chinese(k)
        saying = cut_chinese(ask)
        if to_match(pattern, saying):
            got_patterns = pat_match_with_seg(pattern, saying)
            
            vv = random.choice(v)
            response = ''.join(subsitite(cut_chinese(vv), pat_to_dict2(got_patterns)))
            print(response)
            break

In [35]:
ask_response()

input:你好呀
有事么
